## A demo using Hydrogen Hamiltonian with GPT-QE.

In [23]:
import torch
from gqe.mingpt.utils import set_seed

set_seed(3407)

In [24]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 3.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
# time_pool = [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2]
time_pool = [1 / (2 ** j) for j in range(2, 12)]
time_pool.extend([-1 / (2 ** j) for j in range(2, 12)])

cost = EnergyCost(hamiltonian, initializer, pool, time_pool)


converged SCF energy = -15.0242100060364
paulis [+IIIXIIIIIIII, +IIIXZYIIIIII, +IIIXZZZYIIII, +IIIXZZZZZYII, +IIIXZZZZZZZY, +IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXIIIIIIIII, +IIXXIIIIIIXY, +IIXXIIIIIIYX, +IIXXIIIIXYII, +IIXXIIIIYXII, +IIXXIIXYIIII, +IIXXIIYXIIII, +IIXXXYIIIIII, +IIXXYXIIIIII, +IIXYIIIIIIXX, +IIXYIIIIIIYY, +IIXYIIIIXXII, +IIXYIIIIYYII, +IIXYIIXXIIII, +IIXYIIYYIIII, +IIXYXXIIIIII, +IIXYYYIIIIII, +IIXZYIIIIIII, +IIXZZZYIIIII, +IIXZZZZZYIII, +IIXZZZZZZZYI, +IIYXIIIIIIXX, +IIYXIIIIIIYY, +IIYXIIIIXXII, +IIYXIIIIYYII, +IIYXIIXXIIII, +IIYXIIYYIIII, +IIYXXXIIIIII, +IIYXYYIIIIII, +IIYYIIIIIIXY, +IIYYIIIIIIYX, +IIYYIIIIXYII, +IIYYIIIIYXII, +IIYYIIXYIIII, +IIYYIIYXIIII, +IIYYXYIIIIII, +IIYYYXIIIIII, +IIYZXIIIIIII, +IIYZZZXIIIII, +IIYZZZZZXIII, +IIYZZZZZZZXI, +IXIIIIIIIIII, +IXXIIXZZZZYI, +IXXIIYZZZZXI, +IXXIXZZZZZZY, +IXXIYZZZZZZX, +IXYIIXZZZZXI, +IXYIIYZZZZYI, +IXYIXZZZZZZX, +IXYIYZZZZZZY, +IXZXIXZZZZZY, +IXZXIYZZZZZX, +IXZYIXZZZZZX, +IXZYIYZZZZZY, +IXZZZ

## FCI energy by diagonalization

In [25]:
from qwrapper.hamiltonian import compute_ground_state

#print(compute_ground_state(hamiltonian))

In [26]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -15.024210006036476


## Setup for GPT

In [27]:
# create a GPT instance
from gqe.mingpt.model import GPT

num_layers = 8



In [28]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer
from gqe.mingpt.layer import LayerWiseFineTuneTrainer, LayerWiseFactory
from qwrapper.optimizer import AdamOptimizer
from math import pi

pool = DefaultOperatorPool(paulis)

# time_pool = [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2]

# time_pool = [pi * j / 30 for j in range(0, 30)]
# time_pool.extend([-pi * j / 30 for j in range(0, 30)])


time_pool = [1 / (2 ** j) for j in range(2, 10)]
time_pool.extend([-1 / (2 ** j) for j in range(2, 10)])


class CustomFactory(LayerWiseFactory):
    def generate_train_config(self, layer):
        train_config = Trainer.get_default_config()
        train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
        train_config.max_iters = 200
        train_config.num_workers = 5
        train_config.n_samples = 50
        return train_config

    def generate_model_config(self, layer):
        model_config = GPT.get_default_config()
        model_config.model_type = 'gpt2'
        model_config.vocab_size = pool.size() * len(time_pool)
        model_config.n_gates = 10  # The number of gates for each circuit
        model_config.block_size = model_config.n_gates
        model_config.temperature = 10  # Each gate is generated with probability exp(-temperature * logit)
        model_config.embd_pdrop = 0
        model_config.resid_pdrop = 0
        model_config.attn_pdrop = 0
        model_config.std = 0.02
        model_config.energy_offset = 14
        return model_config


trainer = LayerWiseFineTuneTrainer(CustomFactory(),
                                   hamiltonian=hamiltonian,
                                   initializer=initializer, optimizer=AdamOptimizer(maxiter=300),
                                   num_layers=num_layers, device=get_device(), pool=pool, taus=time_pool)

In [ ]:
print_monitors = []
file_monitors = []
for index in range(num_layers):
    file_monitor = FileMonitor()
    file_monitors.append(file_monitor)
    trainer.set_monitors(index, [PrintMonitor(), file_monitor])
trainer.run()
#torch.save(model.state_dict(), '../saved_models/gptqe_test_2')

layer: 1 starts running
number of parameters: 86.89M
running on device mps


In [ ]:
indices = []
for fm in file_monitors:
    print(fm.min_energy)
    indices.extend(fm.min_indices)
print(trainer.vqe.value())